# TYTX Base - JSON and XML

This notebook demonstrates JSON and XML encoding/decoding with TYTX Base.

## Setup

In [ ]:
from datetime import date, datetime, time
from decimal import Decimal

from genro_tytx import (
    to_typed_text, from_text,
    to_typed_json, from_json,
    to_xml, from_xml,
)

## JSON Encoding

TYTX encodes non-native types with `::suffix` syntax.

In [ ]:
# Sample data with typed values
invoice = {
    "id": "INV-001",
    "total": Decimal("999.99"),
    "date": date(2025, 1, 15),
    "items": [
        {"name": "Widget", "price": Decimal("100.00"), "qty": 2},
        {"name": "Gadget", "price": Decimal("200.00"), "qty": 1},
    ],
}

# Encode to TYTX JSON (text format)
text_encoded = to_typed_text(invoice)
print("Text format:")
print(text_encoded)

In [ ]:
# Encode to TYTX JSON (with protocol prefix)
json_encoded = to_typed_json(invoice)
print("JSON format (with TYTX:// prefix):")
print(json_encoded)

## JSON Decoding

In [ ]:
# Decode text format
decoded = from_text(text_encoded)
print("Decoded from text format:")
print(decoded)
print(f"\nTotal type: {type(decoded['total'])}")
print(f"Date type: {type(decoded['date'])}")

In [ ]:
# Decode JSON format
decoded = from_json(json_encoded)
print("Decoded from JSON format:")
print(decoded)

In [ ]:
# Verify roundtrip
print(f"Roundtrip OK: {decoded == invoice}")

## XML Encoding

In XML, all values are strings, so all types use inline `::suffix` syntax (same as JSON).

In [ ]:
# Simple data
data = {
    "price": Decimal("99.99"),
    "date": date(2025, 1, 15),
    "count": 42,
    "active": True,
}

# Encode to XML
xml = to_xml(data)
print("XML output:")
print(xml)

## XML Decoding

In [ ]:
# Decode XML
decoded = from_xml(xml)
print("Decoded from XML:")
print(decoded)
print(f"\nPrice type: {type(decoded['price'])}")
print(f"Date type: {type(decoded['date'])}")
print(f"Count type: {type(decoded['count'])}")
print(f"Active type: {type(decoded['active'])}")

## XML vs JSON Type Encoding

Both JSON and XML use inline `::suffix` syntax for typed values:

| Type | JSON | XML |
|------|------|-----|
| int | Native | `<count>42::L</count>` |
| float | Native | `<value>3.14::R</value>` |
| bool | Native | `<flag>1::B</flag>` |
| str | Native | (no suffix) |
| Decimal | `"99.99::N"` | `<price>99.99::N</price>` |
| date | `"2025-01-15::D"` | `<date>2025-01-15::D</date>` |
| datetime | `"2025-01-15T10:30:00Z::DHZ"` | `<dt>2025-01-15T10:30:00Z::DHZ</dt>` |
| time | `"10:30:00::H"` | `<time>10:30:00::H</time>` |

In [ ]:
# Compare JSON and XML for same data
data = {"count": 42, "price": Decimal("99.99")}

print("JSON:")
print(to_typed_text(data))
print("\nXML:")
print(to_xml(data))

## Nested XML Structures

In [ ]:
# Nested structure
order = {
    "order": {
        "customer": "Acme Corp",
        "total": Decimal("500.00"),
        "items": [
            {"name": "Widget", "price": Decimal("100.00")},
            {"name": "Gadget", "price": Decimal("200.00")},
        ],
    }
}

xml = to_xml(order)
print("Nested XML:")
print(xml)

In [ ]:
# Decode nested XML
decoded = from_xml(xml)
print("Decoded nested XML:")
print(decoded)

## Receive-Only Types

Some type codes are only decoded, never encoded (because they're native JSON types):

In [ ]:
# L = int (Long)
result = from_text('{"count": "42::L"}::JS')
print(f"L (int): {result}, type: {type(result['count'])}")

# R = float (Real)
result = from_text('{"value": "3.14::R"}::JS')
print(f"R (float): {result}, type: {type(result['value'])}")

# B = bool
result = from_text('{"flag": "1::B"}::JS')
print(f"B (bool): {result}, type: {type(result['flag'])}")

# T = str (Text)
result = from_text('{"name": "hello::T"}::JS')
print(f"T (str): {result}, type: {type(result['name'])}")

## Summary

- **JSON**: Uses `::suffix` inline syntax for non-native types
- **XML**: Uses `::suffix` inline syntax for all types (everything is string in XML)
- **Roundtrip**: Both formats preserve types perfectly